# Tagging genes with ddlite: candidate extraction

## Introduction
In this example **ddlite** app, we'll build a gene tagger from scratch. Here's why we developed ddlite:

* To provide a lighter-weight interface to structured information extraction for new DeepDive users
* To help advanced DeepDive rapidly develop and prototype applications and labeling functions/labelers
* To investigate DeepDive's data programming approach to building inference systems

This example is centered around the second item. Domain-specific tagging systems take months or years to develop. They use hand-crafted model circuitry and accurate, hand-labeled training data. We're going to try to build a pretty good one in a few minutes with none of those things. The generalized extraction and learning utilities provided by ddlite will allow us to turn a sampling of article abstracts and some basic domain knowledge into an automated tagging system. Specifically, we want an accurate tagger for genes in academic articles. We have comprehensive dictionaries of genes, but applying a simple matching rule might yield a lot of false positives. For example, "p53" might get tagged as a gene if it refers to a page number. Our goal is to use distant supervision to improve precision.

Here's the pipeline we'll follow:

1. Obtain and parse input data (relevant article abstracts from PubMed)
2. Extract candidates for tagging
3. Generate features
4. Write labeling functions
5. Learn the tagging model
6. Iterate on labeling functions

Parts 1 and 2 are covered in this notebook, and parts 3 through 6 are covered in `GeneTaggerExample_Learning.ipynb`. Let's get to it.

In [1]:
%load_ext autoreload
%autoreload 2

import cPickle
from ddlite import *

## Processing the input data
We already downloaded the raw HTML for 150 gene-related article pages from PubMed using the `pubmed_gene_html.py` script. These can be found in the `data` folder. We can use ddlite's `DocParser` to read in the article text. There's a general HTML parser which finds visible text, but we can do better by writing a more specific version to just grab the abstract text.

In [2]:
class PubMedAbstractParser(HTMLParser):
    def _cleaner(self, s):
        return (s.parent.name == 'abstracttext')

dp = DocParser('gene_tag_example/data/', PubMedAbstractParser())
docs = list(dp.parseDocs())
print docs[0]

By constructing DNA probes we have identified and cloned a human PtdIns 4-kinase, PI4K230, corresponding to a mRNA of 7.0 kb. The cDNA encodes a protein of 2044 amino acids. The C-terminal part of ca. 260 amino acids represents the catalytic domain which is highly conserved in all recently cloned PtdIns 4-kinases. N-terminal motifs indicate multiple heterologous protein interactions. Human PtdIns 4-kinase PI4K230 expressed in vitro exhibits a specific activity of 58 micromol mg-1min-1. The enzyme expressed in Sf9 cells is essentially not inhibited by adenosine, it shows a high Km for ATP of about 300 microM and it is half-maximally inactivated by approximately 200 nM wortmannin. These data classify this enzyme as type 3 PtdIns 4-kinase. Antibodies raised against the N-terminal part moderately activate and those raised against the C-terminal catalytic domain inhibit the enzymatic activity. The coexistence of two different type 3 PtdIns 4-kinases, PI4K92 and PI4K230, in several human tis

Now we'll use CoreNLP via ddlite's `SentenceParser` to parse each sentence. `DocParser` can handle this too; we didn't really need that call above. This can take a little while, so if the example has already been run, we'll reload it.

In [3]:
docs = None

pkl_f = 'gene_tag_example/gene_tag_saved_sents_v2.pkl'
try:
    with open(pkl_f, 'rb') as f:
        sents = cPickle.load(f)
except:
    %time sents = dp.parseDocSentences()
    with open(pkl_f, 'w+') as f:
        cPickle.dump(sents, f)

print sents[0]

CPU times: user 5.01 s, sys: 126 ms, total: 5.14 s
Wall time: 18.7 s
Sentence(words=['By', 'constructing', 'DNA', 'probes', 'we', 'have', 'identified', 'and', 'cloned', 'a', 'human', 'PtdIns', '4-kinase', ',', 'PI4K230', ',', 'corresponding', 'to', 'a', 'mRNA', 'of', '7.0', 'kb', '.'], lemmas=['by', 'construct', 'dna', 'probe', 'we', 'have', 'identify', 'and', 'clone', 'a', 'human', 'ptdins', '4-kinase', ',', 'pi4k230', ',', 'correspond', 'to', 'a', 'mrna', 'of', '7.0', 'kb', '.'], poses=['IN', 'VBG', 'NN', 'NNS', 'PRP', 'VBP', 'VBN', 'CC', 'VBN', 'DT', 'JJ', 'NN', 'NN', ',', 'NN', ',', 'VBG', 'TO', 'DT', 'NN', 'IN', 'CD', 'NN', '.'], dep_parents=[2, 7, 4, 2, 7, 7, 0, 7, 7, 13, 13, 13, 9, 13, 13, 13, 13, 20, 20, 17, 23, 23, 20, 7], dep_labels=['mark', 'advcl', 'compound', 'dobj', 'nsubj', 'aux', 'ROOT', 'cc', 'conj', 'det', 'amod', 'compound', 'dobj', 'punct', 'appos', 'punct', 'acl', 'case', 'det', 'nmod', 'case', 'nummod', 'nmod', 'punct'], sent_id=0, doc_id=0)


## Extracting candidates with matchers
Extracting candidates for mentions (or relations) in ddlite is done with `Matcher` objects. First, we'll use a `DictionaryMatcher`. We have access to a pretty comprehensive gene dictionary. Let's load it in and create the `DictionaryMatcher`.

In [4]:
# Schema is: ENSEMBL_ID | NAME | TYPE (refseq, canonical, non-canonical)
genes = [line.rstrip().split('\t')[1] for line in open('gene_tag_example/dicts/ensembl_genes.tsv')]
genes = filter(lambda g : len(g) > 2, genes)

dm = DictionaryMatch('GeneName', genes, ignore_case=False)

The dictionary match should provide fairly high recall, but we may still miss some candidates. We know that gene names are named nouns and are often all uppercase. Let's write our own matcher to catch all-uppercase named nouns.

In [5]:
class AllUpperNounsMatcher(Matcher):
    def __init__(self, label):
        self.label = label
        # Regex matcher to find named nouns in part-of-speech tags
        self._re_comp = re.compile("[A-Z]?NN[A-Z]?", flags=re.I)
    def apply(self, s):
        # Get parts-of-speech and words
        words = s.__dict__['words']
        pos = s.__dict__['poses']
        # Get all-cap words
        caps = set(idx for idx, w in enumerate(words) if w.upper() == w)
        # Convert character index to token index
        start_c_idx = [0]
        for s in pos:
            start_c_idx.append(start_c_idx[-1]+len(s)+1)
        # Find regex matches over phrase
        phrase = ' '.join(pos)
        for match in self._re_comp.finditer(phrase):
            # Get start index for tokens
            start = bisect.bisect(start_c_idx, match.start())-1
            # Check if word is capital, has more than two characters, and has a letter
            if start in caps and len(words[start]) > 2 and any(c.isalpha() for c in words[start]):
                yield [start], self.label

up = AllUpperNounsMatcher('UpNoun')

## Creating the candidates
To use candidates from both `Matcher` objects, we can use a `MultiMatcher`. We'll use this to extract our candidate entities from the sentences into an `Entities` object. Using both matchers together will provide very high recall, but may have poor precision. In the next demo notebook (`GeneTaggerExample_Learning.ipynb`), we'll write distant supervision rules and learn a model to improve precision.

In [6]:
M = MultiMatcher(dm, up)
E = Entities(sents, M)

We can visualize contexts for our extractions too. This may help in writing distant supervision rules in `GeneTaggerExample_Learning.ipynb`.

In [7]:
E[0].render()

<!--Provide the canvas id (twice) and the words via python string formatting here--!>
 
 
 By constructing DNA probes we have identified and cloned a human PtdIns 4-kinase , PI4K230 , corresponding to a mRNA of 7.0 kb .

Finally, we'll pickle the extracted candidates from our `Entities` object for use in `GeneTaggerExample_Learning.ipynb`.

In [8]:
E.dump_candidates('gene_tag_example/gene_tag_saved_entities.pkl')